# Analysis of Movie Industry


insert pretty picture here, sponsored by microsoft, microsoft logo

## Overview

In light of Microsoft's desire to expand their enterprise into the movie sector, we have analyzed various qualities of movie data to help develop some insights to help maximize Microsoft's ROI for their potential startup. 

{PUT SUMMARY OF INSIGHTS HERE}

Since we are both relatively unfamiliar with the business end of the industry, supplementary research on the working of the industry as a whole were considered to help drive and focus our analysis. Access sources HERE.

Data was analyzed from IMDB, Box Office Mojo, and The-Numbers.com . (include hyperlinks)
From each of the above 

## Business Problem

picture?

Highlight specific areas of focus
- start up company
- changing industry
- good strategies {use movie terms: release strategy, dump month, etc}

## Unpacking the Data

*Recognize credentials of sources listed above as credible, good data supply, etc



In [ ]:
########## JAMIE CLEANING CODE ###########

In [1]:
# put imports here

In [2]:
# put read csv here

In [ ]:
########### MOOMEN CLEANING CODE #############

### Relevant Variables

Explain each data set

In [3]:
# show head of dataset 1

In [ ]:
# show head of dataset 2

In [ ]:
# show head of dataset 3

In [ ]:
# show head of dataset 4

In [ ]:
# show head of dataset 5

In [ ]:
# show head of dataset 6

## Cleaning / Reorganizing Data

Summarize cleaning process

In [ ]:
################ JAMIE CLEANING #################

In [ ]:
################ CHRISTOS CLEANING ################

### Sub-elements of cleaning (such as important functions)

picture to divide the two sections

## Analysis

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import code.visualizations as viz

%matplotlib inline

In [ ]:
#################### JAMIE VISUALIZATIONS ####################


### Analysis of Variable #1

desc here

### Analysis of Variable #2

### Analysis of Variable #3

### Analysis of Variable #4

In [ ]:
################ CHRISTOS VISUALIZATIONS ####################


### Analysis of Variable #1

desc here

### Analysis of Variable #2

### Analysis of Variable #3

### Analysis of Variable #4

# Conclusions

summaries in nice bullet points 

# Next Steps

next steps